In [ ]:
# Librerie Generiche

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from time import time
import seaborn as sn

# Keras
import keras
from keras.utils import to_categorical

# Sklearn
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,  RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.estimator_checks import check_estimator

np.random.seed(1234)
tf.random.set_seed(1234)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Music = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROGETTO MUSICA DSIM/DATASET/Music_datatset.csv')

In [ ]:
Music= Music.drop(columns=['Unnamed: 0'])
Music.head()

,filename,Mean_ZCR,Var_ZCR,Mean_Harmonic,Var_Harmonic,Mean_Percussive,Var_Percussive,BMP,Mean_Chroma,Var_Chroma,Mean_RMSE,Var_RMSE,Mean_Centroid,Var_Centroid,Mean_Bandwidth,Var_Bandwidth,Mean_Rolloff,Var_Rolloff,Mean_MFCC_1,Mean_MFCC_2,Mean_MFCC_3,Mean_MFCC_4,Mean_MFCC_5,Mean_MFCC_6,Mean_MFCC_7,Mean_MFCC_8,Mean_MFCC_9,Mean_MFCC_10,Mean_MFCC_11,Mean_MFCC_12,Mean_MFCC_13,Mean_MFCC_14,Mean_MFCC_15,Mean_MFCC_16,Mean_MFCC_17,Mean_MFCC_18,Mean_MFCC_19,Mean_MFCC_20,Var_MFCC_1,Var_MFCC_2,Var_MFCC_3,Var_MFCC_4,Var_MFCC_5,Var_MFCC_6,Var_MFCC_7,Var_MFCC_8,Var_MFCC_9,Var_MFCC_10,Var_MFCC_11,Var_MFCC_12,Var_MFCC_13,Var_MFCC_14,Var_MFCC_15,Var_MFCC_16,Var_MFCC_17,Var_MFCC_18,Var_MFCC_19,Var_MFCC_20,Genre
0,blues.00000.wav,0.083045,0.000767,-4.529727e-05,0.008172,0.000008,0.005698,123.046875,0.361408,0.090760,0.130228,0.002827,1784.165850,129774.064525,1999.776906,85884.152948,3776.112794,8.969394e+05,-113.570650,121.571798,-19.168142,42.366419,-6.364663,18.623498,-13.704890,15.343150,-12.274108,10.976571,-8.326572,8.803791,-3.672299,5.747995,-5.162881,0.752739,-1.690214,-0.408980,-2.303522,1.221290,2564.207446,295.913815,235.574441,151.106893,167.934815,89.180847,67.660503,68.932585,82.204203,63.386311,61.773090,51.244127,41.217418,40.554481,49.775427,52.420913,36.524074,41.597104,55.062926,46.936036,Blues
1,blues.00001.wav,0.056040,0.001448,1.395807e-04,0.005099,-0.000178,0.003063,67.999589,0.347103,0.094256,0.095948,0.002373,1530.176679,375850.073649,2022.231367,219372.453803,3450.333583,3.070002e+06,-207.501685,123.991259,8.955128,35.877647,2.907321,21.510467,-8.560437,23.370686,-10.099661,11.900497,-5.555638,5.376327,-2.231760,4.220140,-6.012149,0.927997,-0.731124,0.295072,-0.283518,0.531216,7764.555963,560.259937,572.810924,264.506154,279.932952,156.477119,200.849193,142.555948,166.108518,104.358607,105.173628,96.197214,64.914297,73.152540,52.422137,55.356407,60.314532,48.120599,51.106198,45.786282,Blues
2,blues.00002.wav,0.076291,0.001007,2.105559e-06,0.016342,-0.000019,0.007458,161.499023,0.368103,0.085845,0.175570,0.002746,1552.811865,156467.643368,1745.242239,76376.399848,3014.948204,7.824146e+05,-90.722595,140.446306,-29.093890,31.684336,-13.984503,25.764743,-13.664875,11.639933,-11.783642,9.718760,-13.133803,5.791199,-8.907628,-1.077000,-9.229274,2.451689,-7.729093,-1.816408,-3.439720,-2.231259,3319.044878,508.765036,411.781207,144.090341,155.493772,74.548399,106.981838,106.574869,65.447952,67.908861,57.781425,64.480209,60.385150,57.711139,36.580990,40.598769,47.639430,52.382139,46.639666,30.573022,Blues
3,blues.00003.wav,0.033309,0.000423,4.583532e-07,0.019054,-0.000014,0.002712,63.024009,0.404718,0.093024,0.141093,0.006346,1070.106615,184355.942417,1582.351010,167302.429241,2124.259648,1.471924e+06,-199.544200,150.090908,5.662679,26.859079,1.771400,14.234030,-4.832006,9.286495,-0.759186,8.137606,-3.200652,6.079319,-2.480173,-1.079305,-2.870788,0.780873,-3.319597,0.636964,-0.619121,-3.407449,5507.517290,456.505368,257.161163,158.267309,268.034409,126.794130,155.912087,81.273747,92.114088,71.314079,110.236679,48.251998,56.799406,62.289909,51.651593,44.427757,50.206674,37.319132,37.259741,31.949340,Blues
4,blues.00004.wav,0.101461,0.001954,-1.756132e-05,0.004814,-0.000010,0.003094,135.999178,0.313369,0.088829,0.091529,0.002303,1835.004266,343399.939274,1745.887188,90657.194397,3524.575671,1.612968e+06,-160.337711,126.219631,-35.587809,22.148071,-32.478602,10.852293,-23.352330,0.498432,-11.793436,1.207254,-13.088417,-2.813868,-6.933384,-7.552726,-9.164665,-4.520577,-5.454034,-0.916875,-4.404826,-11.703234,5195.292192,853.784702,333.792979,193.456118,336.276843,134.831603,93.257102,124.672134,130.073373,99.675591,80.254066,86.430623,89.555446,70.943335,75.793402,86.099247,75.269718,53.613917,62.910815,55.195162,Blues


In [ ]:
data_X = Music.drop(columns=['filename', 'Genre'])
X = data_X.values
min_max_scaler_x = sklearn.preprocessing.MinMaxScaler()
X_scale = min_max_scaler_x.fit_transform(X)
X_scale

array([[0.242545  , 0.02520928, 0.63277922, ..., 0.08244974, 0.10950252,
        0.07825494],
       [0.1357779 , 0.04894396, 0.63727538, ..., 0.10299716, 0.09908115,
        0.0759467 ],
       [0.21584389, 0.03357507, 0.63393203, ..., 0.11641998, 0.08731703,
        0.04540464],
       ...,
       [0.39585688, 0.0576381 , 0.63386896, ..., 0.06482372, 0.07641965,
        0.04398425],
       [0.10685518, 0.0266249 , 0.63390025, ..., 0.05956411, 0.08886446,
        0.06693823],
       [0.21589764, 0.0741852 , 0.63389202, ..., 0.19239899, 0.15461065,
        0.08372859]])

In [ ]:
data_Y = Music['Genre']
Y = data_Y.values
LE = sklearn.preprocessing.LabelEncoder()
Y_label = LE.fit_transform(Y)

In [ ]:
X_scale.shape

(999, 57)

In [ ]:
Y_label.shape

(999,)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_scale, Y_label, test_size=0.2)

In [ ]:
X_train.shape

(799, 57)

In [ ]:
X_test.shape

(200, 57)

In [ ]:
y_train.shape

(799,)

In [ ]:
y_test.shape

(200,)

In [ ]:
y_train[100]

2

In [ ]:
# SVM

# Random Search

# Parameters to be tested in cross-validation
param_grid = {'C': [1e2, 5e2, 1e3],
          'gamma': [0.005, 0.01, 0.1, 0.5, 1.0], }

# Support Vector Machine initialization
clf_SVC = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid, cv=2)

# Training
t0 = time()
clf_SVC = clf_SVC.fit(X_train, y_train)
print('Training completed in %0.3fs' % (time() - t0))
# Test
y_pred_SVC = clf_SVC.predict(X_test)

# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_SVC))

# Confusion matrix
print('Confusion matrix:')
cm = confusion_matrix(y_test, y_pred_SVC)
print(cm)

Training completed in 1.090s
Classification report:
              precision    recall  f1-score   support

           0       0.74      0.94      0.83        18
           1       0.81      0.91      0.86        23
           2       0.90      0.75      0.82        24
           3       0.76      0.64      0.70        25
           4       0.88      0.62      0.73        24
           5       0.85      0.73      0.79        15
           6       0.86      0.86      0.86        21
           7       0.60      0.80      0.69        15
           8       0.65      0.65      0.65        17
           9       0.64      0.78      0.70        18

    accuracy                           0.77       200
   macro avg       0.77      0.77      0.76       200
weighted avg       0.78      0.77      0.76       200

Confusion matrix:
[[17  0  1  0  0  0  0  0  0  0]
 [ 0 21  1  0  0  1  0  0  0  0]
 [ 0  0 18  1  0  1  0  2  0  2]
 [ 1  1  0 16  0  0  0  3  1  3]
 [ 2  0  0  1 15  0  2  2  2  0]
 [ 0  

In [ ]:
# Result of the cross validation for parameters selection SVM
print('Best parameters combination:')
print(' C: '+str(clf_SVC.best_estimator_.C))
print(' gamma: '+str(clf_SVC.best_estimator_.gamma))

Best parameters combination:
 C: 100.0
 gamma: 1.0


In [ ]:
# LOGISTIC REGRESSION

# Random Search

space = dict()
space['solver'] = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
space['C'] = (range(1,1000))
space['multi_class'] = ['auto', 'ovr', 'multinomial']
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = LogisticRegression()

clf_LR = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_LR = clf_LR.fit(X_train, y_train)

# Pred
y_pred_LR = clf_LR.predict(X_test)

# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_LR))

# Confusion matrix
print('Confusion matrix:')
cm_LR = confusion_matrix(y_test, y_pred_LR)
print(cm_LR)

Classification report:
              precision    recall  f1-score   support

           0       0.94      0.83      0.88        18
           1       0.83      0.83      0.83        23
           2       0.72      0.54      0.62        24
           3       0.55      0.44      0.49        25
           4       0.90      0.79      0.84        24
           5       0.73      0.73      0.73        15
           6       0.81      0.81      0.81        21
           7       0.72      0.87      0.79        15
           8       0.67      0.71      0.69        17
           9       0.40      0.67      0.50        18

    accuracy                           0.71       200
   macro avg       0.73      0.72      0.72       200
weighted avg       0.73      0.71      0.71       200

Confusion matrix:
[[15  0  0  0  0  0  1  0  2  0]
 [ 0 19  1  0  0  2  0  0  0  1]
 [ 0  1 13  3  0  0  0  2  0  5]
 [ 0  1  1 11  1  0  0  2  2  7]
 [ 0  0  0  1 19  0  3  1  0  0]
 [ 0  2  1  0  0 11  0  0  1  0]
 [

In [ ]:
# Result of the cross validation for parameters selection Logistic Regression
print('Best parameters combination:')
print(' C: '+str(clf_LR.best_estimator_.C))
print(' Penalty: '+str(clf_LR.best_estimator_.penalty))
print(' Solver: '+str(clf_LR.best_estimator_.solver))
print(' Solver: '+str(clf_LR.best_estimator_.multi_class))

Best parameters combination:
 C: 614
 Penalty: l2
 Solver: saga
 Solver: multinomial


In [ ]:
# Random Forrest

# Random Search

space = dict()
space['n_estimators'] = (range(0,1000))
space['criterion']= ['gini', 'entropy']
space['min_samples_split'] = (range(2,10))
space['min_samples_leaf'] = (range(1,10))
space['max_features'] = ["auto", "sqrt", "log2"]
space['bootstrap'] = ['True', 'False']
space['oob_score'] = ['True', 'False']
space['warm_start'] = ['True', 'False']

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = RandomForestClassifier()

clf_RF = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_RF = clf_RF.fit(X_train, y_train)

# Pred
y_pred_RF = clf_RF.predict(X_test)

# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_RF))

# Confusion matrix
print('Confusion matrix:')
cm_RF = confusion_matrix(y_test, y_pred_RF)
print(cm_RF)

Classification report:
              precision    recall  f1-score   support

           0       0.71      0.67      0.69        18
           1       0.77      0.74      0.76        23
           2       0.68      0.54      0.60        24
           3       0.62      0.52      0.57        25
           4       0.84      0.67      0.74        24
           5       0.56      0.60      0.58        15
           6       0.67      0.95      0.78        21
           7       0.72      0.87      0.79        15
           8       0.60      0.53      0.56        17
           9       0.30      0.39      0.34        18

    accuracy                           0.65       200
   macro avg       0.65      0.65      0.64       200
weighted avg       0.66      0.65      0.64       200

Confusion matrix:
[[12  0  2  0  0  1  2  0  0  1]
 [ 0 17  1  0  0  5  0  0  0  0]
 [ 1  0 13  2  0  1  0  0  1  6]
 [ 0  1  1 13  2  0  1  2  0  5]
 [ 1  0  0  1 16  0  3  1  2  0]
 [ 0  4  1  0  0  9  0  0  1  0]
 [

In [ ]:
# Result of the cross validation for parameters selection Random Forrest
print('Best parameters combination:')
print(' n_estimators: '+str(clf_RF.best_estimator_.n_estimators))
print(' criterion: '+str(clf_RF.best_estimator_.criterion))
print(' min_samples_split: '+str(clf_RF.best_estimator_.min_samples_split))
print(' min_samples_leaf: '+str(clf_RF.best_estimator_.min_samples_leaf))
print(' max_features: '+str(clf_RF.best_estimator_.max_features))
print(' bootstrap: '+str(clf_RF.best_estimator_.bootstrap))
print(' oob_score: '+str(clf_RF.best_estimator_.oob_score))
print(' warm_start: '+str(clf_RF.best_estimator_.warm_start))

Best parameters combination:
 n_estimators: 519
 criterion: entropy
 min_samples_split: 5
 min_samples_leaf: 1
 max_features: auto
 bootstrap: True
 oob_score: False
 warm_start: False


In [ ]:
# Decision Tree

# Random Search

space = dict()
space['splitter'] = ['best', 'random']
space['criterion']= ['gini', 'entropy']
space['min_samples_split'] = (range(2,100))
space['min_samples_leaf'] = (range(1,100))
space['max_features'] = ["auto", "sqrt", "log2"]

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = DecisionTreeClassifier()

clf_DT = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_DT = clf_DT.fit(X_train, y_train)

# Pred
y_pred_DT = clf_DT.predict(X_test)

# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_DT))

# Confusion matrix
print('Confusion matrix:')
cm_DT = confusion_matrix(y_test, y_pred_DT)
print(cm_DT)


Classification report:
              precision    recall  f1-score   support

           0       0.44      0.22      0.30        18
           1       0.76      0.57      0.65        23
           2       0.33      0.21      0.26        24
           3       0.26      0.20      0.23        25
           4       0.54      0.62      0.58        24
           5       0.36      0.67      0.47        15
           6       0.58      0.90      0.70        21
           7       0.41      0.73      0.52        15
           8       0.50      0.41      0.45        17
           9       0.10      0.06      0.07        18

    accuracy                           0.45       200
   macro avg       0.43      0.46      0.42       200
weighted avg       0.43      0.45      0.42       200

Confusion matrix:
[[ 4  0  6  1  1  3  2  0  1  0]
 [ 0 13  0  0  0  7  0  1  0  2]
 [ 2  1  5  7  1  1  0  3  3  1]
 [ 0  0  2  5  6  2  3  4  0  3]
 [ 0  0  0  1 15  0  3  3  1  1]
 [ 1  3  0  1  0 10  0  0  0  0]
 [

In [ ]:
# Result of the cross validation for parameters selection Random Forrest
print('Best parameters combination:')
print(' splitter: '+str(clf_DT.best_estimator_.splitter))
print(' criterion: '+str(clf_DT.best_estimator_.criterion))
print(' min_samples_split: '+str(clf_DT.best_estimator_.min_samples_split))
print(' min_samples_leaf: '+str(clf_DT.best_estimator_.min_samples_leaf))
print(' max_features: '+str(clf_DT.best_estimator_.max_features))

Best parameters combination:
 splitter: best
 criterion: gini
 min_samples_split: 47
 min_samples_leaf: 20
 max_features: auto


In [ ]:
# KNeighbors Classifier

# Random Search

space = dict()
space['n_neighbors'] = range(5,50)
space['algorithm']= ['auto', 'ball_tree', 'kd_tree', 'brute']
space['p'] = [1, 2]

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = KNeighborsClassifier()

clf_KN = RandomizedSearchCV(model, space, scoring='accuracy', n_jobs=-1)
# Training
clf_KN = clf_KN.fit(X_train, y_train)

# Pred
y_pred_KN = clf_KN.predict(X_test)

# Classificaiton report
print('Classification report:')
print(classification_report(y_test, y_pred_KN))

# Confusion matrix
print('Confusion matrix:')
cm_KN = confusion_matrix(y_test, y_pred_KN)
print(cm_KN)

Classification report:
              precision    recall  f1-score   support

           0       0.72      0.72      0.72        18
           1       0.74      0.87      0.80        23
           2       0.59      0.83      0.69        24
           3       0.75      0.72      0.73        25
           4       0.76      0.67      0.71        24
           5       0.80      0.53      0.64        15
           6       0.80      0.95      0.87        21
           7       0.70      0.93      0.80        15
           8       0.44      0.24      0.31        17
           9       0.50      0.33      0.40        18

    accuracy                           0.69       200
   macro avg       0.68      0.68      0.67       200
weighted avg       0.69      0.69      0.68       200

Confusion matrix:
[[13  0  3  0  0  1  0  0  0  1]
 [ 0 20  2  0  0  1  0  0  0  0]
 [ 0  1 20  0  0  0  0  2  0  1]
 [ 0  1  2 18  0  0  1  1  0  2]
 [ 1  0  0  1 16  0  2  2  2  0]
 [ 0  5  1  0  0  8  0  0  1  0]
 [

In [ ]:
# Result of the cross validation for parameters selection Random Forrest
print('Best parameters combination:')
print(' n_neighbors: '+str(clf_KN.best_estimator_.n_neighbors))
print(' algorithm: '+str(clf_KN.best_estimator_.algorithm))
print(' mp: '+str(clf_KN.best_estimator_.p))

Best parameters combination:
 n_neighbors: 6
 algorithm: auto
 mp: 1


In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=128)

Epoch 1/100
7/7 [==============================] - 2s 4ms/step - loss: 2.2499 - accuracy: 0.2074
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 1.9132 - accuracy: 0.3492
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 1.6220 - accuracy: 0.4311
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 1.4068 - accuracy: 0.4705
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 1.2756 - accuracy: 0.5239
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 1.1908 - accuracy: 0.5580
Epoch 7/100
7/7 [==============================] - 0s 2ms/step - loss: 1.1685 - accuracy: 0.5531
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0829 - accuracy: 0.6115
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0106 - accuracy: 0.6318
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0166 - accuracy: 0.6211
Epoch 11/100
7/7 [===========

In [ ]:
test_loss, test_acc  = model.evaluate(X_test, y_test, batch_size=128)

2/2 [==============================] - 0s 4ms/step - loss: 1.4478 - accuracy: 0.7000
